In [1]:
import pandas as pd
import pyodbc
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import sqlalchemy as sa
from datetime import datetime, timedelta

# Load your CSV file into a DataFrame
df = pd.read_csv('data/data.csv').head(1)


In [2]:
#make connection to DeltaDB
connection_string = '''Driver={SQL Server Native Client 11.0}; 
                        Server=SQL-PR_DeltaDB_RFQ-PR,1521; 
                        Database=PR_DeltaDB;
                        Trusted_Connection=yes;'''

connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine_DeltaDB = create_engine(connection_url)

#make connection to DeltaDB
connection_string = '''Driver={SQL Server Native Client 11.0}; 
                        Server=SQL-PR_DeltaDB_RFQ-PR,1521; 
                        Database=PR_DeltaDB_JB;
                        Trusted_Connection=yes;'''

connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine_DeltaDB_JB = create_engine(connection_url)


In [3]:

def fetch_inventory_prev_day_rows(conn, formatted_prev_date):
    query = sa.text(f'''
        SELECT [InstrumentID], [Number]
        FROM [PR_DeltaDB_JB].[dbo].[Position]
        WHERE GETDATE() < Deleted
          AND PositionRule in ('jms04')
          AND ValidFrom = :formatted_prev_date
          order by InstrumentID
    ''')
    
    # Execute the query
    result = pd.read_sql_query(query, conn, params={"formatted_prev_date": formatted_prev_date})
    return result

def fetch_inventory_prev_day_rows_and_currency(conn, formatted_prev_date):
    query = sa.text(f'''
        SELECT distinct P.InstrumentID, P.Number, IRD.CurrencyID
        FROM [PR_DeltaDB_JB].[dbo].[Position] P
            JOIN [PR_DeltaDB].[dbo].[InstrumentRepositoryData] IRD
            ON P.InstrumentID COLLATE SQL_Latin1_General_CP1_CI_AS = IRD.InstrumentID COLLATE SQL_Latin1_General_CP1_CI_AS
        WHERE GETDATE() < P.Deleted
            AND P.PositionRule IN ('jms04')
            AND P.ValidFrom = :formatted_prev_date
            ORDER BY P.InstrumentID;
    ''')
    
    # Execute the query
    result = pd.read_sql_query(query, conn, params={"formatted_prev_date": formatted_prev_date})
    return result




# Function to fetch BPV for a given InstrumentID and TradeTime
def fetch_bpv(conn, instrument_id, trade_time):
    query_bpv = sa.text('''
        SELECT TOP 1 [Number] AS BPV
        FROM [PR_DeltaDB].[dbo].[KeyRatio]
        WHERE GETDATE() < Deleted
          AND InstrumentID = :instrument_id
          AND Context = 'Risikostyring'
          AND KeyRatioName = 'BPV'
          AND ValidFrom < :trade_time
        ORDER BY ValidFrom DESC
    ''')

    result_bpv = pd.read_sql_query(query_bpv, conn, params={'instrument_id': instrument_id, 'trade_time': trade_time})
    
    # If there is a result, return the BPV, otherwise return 0.0 or NaN
    return result_bpv['BPV'].iloc[0] if not result_bpv.empty else 0.0

def fetch_collateral_prev_day(conn, instrument_id, formatted_prev_date):
    query = sa.text(f'''SELECT sum(Number) as collateralSum
            FROM [PR_DeltaDB_JB].[dbo].[Position]
            where GETDATE() < Deleted
            and PositionRule in ('JMS07',
                                    'JMS08',
                                    'JMS14',
                                    'JMS17',
                                    'JMS18'
                                    )
            and ValidFrom = :formatted_prev_day
            and InstrumentID = :InstrumentID
            ''')
                
    # Execute the query with the trade_time as a parameter
    result_df = pd.read_sql_query(query, conn, params={"formatted_prev_day": formatted_prev_date, "InstrumentID": instrument_id})
    value = result_df['collateralSum'].iloc[0] 
    return value if value is not None else 0


def fetch_transactions_current_day_Isin(conn, formatted_prev_date, formatted_trade_time, instrument_id, portfolio):
    query = sa.text(f'''select sum(Number) as currentDaySum
            from [PR_DeltaDB_JB].[dbo].[Transactions]  
            where GETDATE() < Deleted
            and Portfolio = :portfolio
            and InstrumentID = :InstrumentID
            and :formatted_prev_day < Settlement
            and TradeTime <= :formatted_trade_time 
            ''')
    
    # Execute the query with the trade_time as a parameter
    result_df = pd.read_sql_query(query, conn, params={"formatted_prev_day": formatted_prev_date, 
                                                    "formatted_trade_time": formatted_trade_time, 
                                                    "InstrumentID": instrument_id,
                                                    "portfolio": portfolio})
    
    value = result_df['currentDaySum'].iloc[0] 
    return value if value is not None else 0


def fetch_collateral_current_day(conn, formatted_prev_day, formatted_trade_time, instrument_id):
    query = sa.text(f'''select sum(Number) as currentDayCollateralSum
        from [PR_DeltaDB_JB].[dbo].[Transactions]  
        where GETDATE() < Deleted
        and Portfolio in ('JMS07',
						'JMS08',
						'JMS14',
						'JMS17',
						'JMS18'
						)
        and InstrumentID = :InstrumentID
        and :formatted_prev_day < Settlement
        and TradeTime <= :formatted_trade_time 
            ''')
    
    # Execute the query with the trade_time as a parameter
    result_df = pd.read_sql_query(query, conn, params={"formatted_prev_day": formatted_prev_day, 
                                                    "formatted_trade_time": formatted_trade_time, 
                                                    "InstrumentID": instrument_id})
    
    value = result_df['currentDayCollateralSum'].iloc[0] 
    return value if value is not None else 0


def fetch_currency(conn, instrument_id):
    query = sa.text(f'''select distinct CurrencyID 
            from InstrumentRepositoryData
            where InstrumentID = :InstrumentID
            ''')
    result_df = pd.read_sql_query(query, conn, params={ "InstrumentID": instrument_id})
    
    currency = result_df['CurrencyID'].iloc[0] 
    assert currency is not None, "Currency not retrieved"
        
    return currency 

def fetch_new_positions(conn, formatted_prev_date, formatted_trade_time, portfolio):
    query = sa.text(f'''(SELECT InstrumentID
                FROM [PR_DeltaDB_JB].[dbo].[Transactions]
                WHERE GETDATE() < Deleted
                AND Portfolio = :portfolio
                AND :formatted_prev_date < Settlement
                AND TradeTime <= :formatted_trade_time)

                EXCEPT

                (SELECT InstrumentID
                FROM [PR_DeltaDB_JB].[dbo].[Position]
                WHERE GETDATE() < Deleted
                AND PositionRule = :portfolio 
                AND ValidFrom = :formatted_prev_date)''')
    
    result_df = pd.read_sql_query(query, conn, params={"formatted_prev_date": formatted_prev_date, 
                                                    "formatted_trade_time": formatted_trade_time,
                                                    "portfolio": portfolio})
    return result_df



In [4]:
def fetch_inventory_prev_day_rows_and_currency(conn, formatted_prev_date, portfolio):
    query = sa.text(f'''
        SELECT distinct P.InstrumentID, P.Number, IRD.CurrencyID
        FROM [PR_DeltaDB_JB].[dbo].[Position] P
            JOIN [PR_DeltaDB].[dbo].[InstrumentRepositoryData] IRD
            ON P.InstrumentID COLLATE SQL_Latin1_General_CP1_CI_AS = IRD.InstrumentID COLLATE SQL_Latin1_General_CP1_CI_AS
        WHERE GETDATE() < P.Deleted
            AND P.PositionRule = :portfolio
            AND P.ValidFrom = :formatted_prev_date
            ORDER BY P.InstrumentID;
    ''')
    
    # Execute the query
    result = pd.read_sql_query(query, conn, params={"formatted_prev_date": formatted_prev_date, "portfolio": portfolio})
    return result


def fetch_bpv_bulk(conn, instrument_ids, trade_time):
    # Format the list of instrument IDs to be used in the SQL query
    instrument_ids_placeholder = ', '.join([f"'{isin}'" for isin in instrument_ids])
    
    query_bpv = sa.text(f'''
        WITH RankedBPV AS (
            SELECT InstrumentID, Number AS BPV, 
                   ROW_NUMBER() OVER (PARTITION BY InstrumentID ORDER BY ValidFrom DESC) AS rn
            FROM [PR_DeltaDB].[dbo].[KeyRatio]
            WHERE GETDATE() < Deleted
              AND Context = 'Risikostyring'
              AND KeyRatioName = 'BPV'
              AND ValidFrom < :trade_time
              AND InstrumentID IN ({instrument_ids_placeholder})
        )
        SELECT InstrumentID, BPV
        FROM RankedBPV
        WHERE rn = 1
        order by InstrumentID
    ''')

    # Execute the query and return the result as a dictionary
    result_df = pd.read_sql_query(query_bpv, conn, params={'trade_time': trade_time})
    return (result_df.set_index('InstrumentID')['BPV']/100).to_dict()

def fetch_collateral_prev_day_bulk(conn, instrument_ids, formatted_prev_date):
    # Format the list of instrument IDs to be used in the SQL query
    instrument_ids_placeholder = ', '.join([f"'{isin}'" for isin in instrument_ids])
    
    query = sa.text(f'''
        SELECT InstrumentID, sum(Number) as collateralSum
        FROM [PR_DeltaDB_JB].[dbo].[Position]
        WHERE GETDATE() < Deleted
          AND PositionRule IN ('JMS07', 'JMS08', 'JMS14', 'JMS17', 'JMS18')
          AND ValidFrom = :formatted_prev_day
          AND InstrumentID IN ({instrument_ids_placeholder})
        GROUP BY InstrumentID
    ''')

    result_df = pd.read_sql_query(query, conn, params={'formatted_prev_day': formatted_prev_date})
    return result_df.set_index('InstrumentID')['collateralSum'].to_dict()


def fetch_transactions_current_day_bulk(conn, instrument_ids, formatted_prev_date, formatted_trade_time, portfolio):
    # Format the list of instrument IDs to be used in the SQL query
    instrument_ids_placeholder = ', '.join([f"'{isin}'" for isin in instrument_ids])
    
    
    query = sa.text(f'''
        SELECT InstrumentID, sum(Number) as currentDaySum
        FROM [PR_DeltaDB_JB].[dbo].[Transactions]
        WHERE GETDATE() < Deleted
          AND Portfolio = :portfolio
          AND InstrumentID IN ({instrument_ids_placeholder})
          AND :formatted_prev_day < Settlement
          AND TradeTime <= :formatted_trade_time
        GROUP BY InstrumentID
    ''')

    result_df = pd.read_sql_query(query, conn, params={'formatted_prev_day': formatted_prev_date, 
                                                       'formatted_trade_time': formatted_trade_time,
                                                       'portfolio': portfolio })
    
    return result_df.set_index('InstrumentID')['currentDaySum'].to_dict()

def fetch_collateral_current_day_bulk(conn, instrument_ids, formatted_prev_day, formatted_trade_time):
    # Format the list of instrument IDs to be used in the SQL query
    instrument_ids_placeholder = ', '.join([f"'{isin}'" for isin in instrument_ids])
    
    query = sa.text(f'''
        SELECT InstrumentID, sum(Number) as currentDayCollateralSum
        FROM [PR_DeltaDB_JB].[dbo].[Transactions]
        WHERE GETDATE() < Deleted
          AND Portfolio IN ('JMS07', 'JMS08', 'JMS14', 'JMS17', 'JMS18')
          AND InstrumentID IN ({instrument_ids_placeholder})
          AND :formatted_prev_day < Settlement
          AND TradeTime <= :formatted_trade_time
        GROUP BY InstrumentID
    ''')

    result_df = pd.read_sql_query(query, conn, params={'formatted_prev_day': formatted_prev_day, 
                                                       'formatted_trade_time': formatted_trade_time})
    return result_df.set_index('InstrumentID')['currentDayCollateralSum'].to_dict()

Risk BPV measure

In [5]:
df['inventory_Risk'] = 0.0
df['bpv_risk'] = 0.0

# Open a connection using the SQLAlchemy engine
with engine_DeltaDB_JB.begin() as conn_DB_JB, engine_DeltaDB.begin() as conn_DB:
        
        # Iterate over the rows in the bond trades dataframe
        for index, row in df.iterrows():
            trade_time = row['TradeTime']
            
            # Parse the dates into a datetime objects and strings
            date_object = datetime.strptime(trade_time, '%m/%d/%Y %H:%M:%S')

            trade_time_formatted = date_object.strftime('%Y-%m-%d %H:%M:%S') 
            
            prev_day = date_object - timedelta(days=1)

            # Convert the datetime object to the desired format
            formatted_prev_date = prev_day.strftime('%Y-%m-%d')

            ##########################################################################################

            trade_time_formatted = '2024-10-03 23:00'
            formatted_prev_date =  '2024-10-02'
            
            
            # Initialize the result variable
            inventory_previous_day = fetch_inventory_prev_day_rows(conn_DB_JB, formatted_prev_date)
            
            # While loop to keep subtracting days until a valid result is found
            while inventory_previous_day.empty:
                # Subtract one more day
                prev_day -= timedelta(days=1)
                formatted_prev_date = prev_day.strftime('%Y-%m-%d')

                # Fetch inventory data again for the updated previous day
                inventory_previous_day = fetch_inventory_prev_day_rows(conn_DB_JB, formatted_prev_date)

            
            print('tradetime: ', trade_time_formatted)
            print('prev date: ', formatted_prev_date)
            
            ################################################################################################
            
            # Now that we have a valid result, update the DataFrame
            inventory = 0
            bpv_risk = 0

            for idx, row in inventory_previous_day.iterrows():
                    
                    instrument_id = row['InstrumentID']
                    if instrument_id.startswith('DK'):
                        number_prev_day = row['Number']
                        collateral_prev_day = fetch_collateral_prev_day(conn_DB_JB, instrument_id, formatted_prev_date)
                        number_curr_day = fetch_transactions_current_day_Isin(conn_DB_JB, formatted_prev_date, trade_time_formatted, instrument_id)
                        collateral_curr_day = fetch_collateral_current_day(conn_DB_JB, formatted_prev_date, trade_time_formatted, instrument_id)
                        bpv = fetch_bpv(conn_DB, instrument_id, trade_time_formatted)/100
                        currency = fetch_currency(conn_DB, instrument_id)
                        currency_multiplier = 7.45 if currency == 'EUR' else 1.0
                        risk_inventory_isin = (number_prev_day+collateral_prev_day + number_curr_day+collateral_curr_day)*currency_multiplier
                        inventory += risk_inventory_isin
                        bpv_risk += risk_inventory_isin*bpv
                        #print(instrument_id, collateral_curr_day)
                        
                        #print(f'''ID: {instrument_id}, numberPrevDay: {number_prev_day}, collateral prev day: {collateral_prev_day}, 
                        #      number curr day {number_curr_day}, collateral curr day: {collateral_curr_day}, BPV_risk: {risk_inventory_isin*bpv}''')
                        
                        #data.append({
                        #    'InstrumentID': instrument_id,
                        #    'risk_inventory_isin': risk_inventory_isin,
                        #    'BPV_risk': risk_inventory_isin*bpv
                        #})
            
            new_positions = fetch_new_positions(conn_DB_JB, formatted_prev_date, trade_time_formatted)
            isin_list = new_positions['InstrumentID'].to_list()
            for isin in isin_list:
                number_curr_day = fetch_transactions_current_day_Isin(conn_DB_JB, formatted_prev_date, trade_time_formatted, isin)
                collateral_curr_day = fetch_collateral_current_day(conn_DB_JB, formatted_prev_date, trade_time_formatted, isin)
                currency = fetch_currency(conn_DB, isin)
                bpv = fetch_bpv(conn_DB, isin, trade_time_formatted)/100
                currency_multiplier = 7.45 if currency == 'EUR' else 1.0
                risk_inventory_isin = (number_curr_day+collateral_curr_day)*currency_multiplier
                inventory += risk_inventory_isin
                bpv_risk += risk_inventory_isin*bpv
            
            '''
            isins = inventory_previous_day.InstrumentID.to_list()
            bpv_dict = fetch_bpv_bulk2(conn_DB_JB, isins, trade_time_formatted) 
            #print(bpv_dict)

            collateral_prev_dict = fetch_collateral_prev_day_bulk(conn_DB_JB, isins, formatted_prev_date)
            #print(collateral_prev_dict)

            trans_curr_dict = fetch_transactions_current_day_bulk(conn_DB_JB, isins, formatted_prev_date, trade_time_formatted)
            #print(trans_curr_dict)

            collateral_curr_dict = fetch_collateral_current_day_bulk(conn_DB_JB, isins, formatted_prev_date, trade_time_formatted)
            print(collateral_curr_dict)'''

            #Append values to original df
            df.at[index, 'inventoryRisk'] = (inventory)/1000
            df.at[index, 'bpv_risk'] = (bpv_risk)/1000

            #df_risk_inventory = pd.DataFrame(data) 

print(df.inventoryRisk)
print(df.bpv_risk)        
            

tradetime:  2024-10-03 23:00
prev date:  2024-10-02


TypeError: fetch_transactions_current_day_Isin() missing 1 required positional argument: 'portfolio'

In [6]:
import numpy as np
df['inventory_Risk'] = 0.0
df['bpv_risk'] = 0.0


# Open a connection using the SQLAlchemy engine
with engine_DeltaDB_JB.begin() as conn_DB_JB, engine_DeltaDB.begin() as conn_DB:
        
        # Iterate over the rows in the bond trades dataframe
        for index, row in df.iterrows():
            trade_time = row['TradeTime']
            portfolio = 'jms04' if row['BookName'] == 'KORTMM' else 'jms05'


            # Parse the dates into a datetime objects and strings
            date_object = datetime.strptime(trade_time, '%m/%d/%Y %H:%M:%S')

            trade_time_formatted = date_object.strftime('%Y-%m-%d %H:%M:%S') 
            
            prev_day = date_object - timedelta(days=1)

            # Convert the datetime object to the desired format
            formatted_prev_date = prev_day.strftime('%Y-%m-%d')

            ##########################################################################################
            
            trade_time_formatted = '2024-10-03 23:00'
            formatted_prev_date =  '2024-10-02'
            portfolio = 'jms04'

            # Initialize the result variable
            inventory_previous_day = fetch_inventory_prev_day_rows_and_currency(conn_DB_JB, formatted_prev_date, portfolio)
            
            # While loop to keep subtracting days until a valid result is found
            while inventory_previous_day.empty:
                # Subtract one more day
                prev_day -= timedelta(days=1)
                formatted_prev_date = prev_day.strftime('%Y-%m-%d')

                # Fetch inventory data again for the updated previous day
                inventory_previous_day = fetch_inventory_prev_day_rows_and_currency(conn_DB_JB, formatted_prev_date, portfolio)

            print('tradetime: ', trade_time_formatted)
            print('prev date: ', formatted_prev_date)
            
            ################################################################################################
            
            # Now that we have a valid result, update the DataFrame
            inventory = 0
            bpv_risk = 0
            data = []

            instrument_ids = inventory_previous_day.InstrumentID.to_list()

            bpv_dict = fetch_bpv_bulk(conn_DB, instrument_ids, trade_time_formatted)
            collateral_prev_day_dict = fetch_collateral_prev_day_bulk(conn_DB_JB, instrument_ids, formatted_prev_date)
            inventory_curr_day_dict = fetch_transactions_current_day_bulk(conn_DB_JB, instrument_ids, formatted_prev_date, trade_time_formatted, portfolio)
            collateral_curr_day_dict = fetch_collateral_current_day_bulk(conn_DB_JB, instrument_ids, formatted_prev_date, trade_time_formatted)


            for idx, row in inventory_previous_day.iterrows():
                    
                isin = row['InstrumentID']
                inventory_prev_day = row['Number']
                currency = row['CurrencyID']  # This could also potentially be batched
                currency_multiplier = 7.45 if currency == 'EUR' else 1.0
                
                collateral_prev_day = collateral_prev_day_dict.get(isin, 0.0)
                inventory_curr_day = inventory_curr_day_dict.get(isin, 0.0)
                collateral_curr_day = collateral_curr_day_dict.get(isin, 0.0)
                bpv = bpv_dict.get(isin, 0.0) 

                # Process risk and inventory calculations
                risk_inventory_isin = (inventory_prev_day + collateral_prev_day + inventory_curr_day + collateral_curr_day) * currency_multiplier
                inventory += risk_inventory_isin
                bpv_risk += risk_inventory_isin*bpv

                #data.append({
                #            'InstrumentID': isin,
                #            'risk_inventory_isin': risk_inventory_isin,
                #            'BPV_risk': risk_inventory_isin*bpv
                #        })
            
            # Add transactions current day for isin not found in Position table 
            new_positions = fetch_new_positions(conn_DB_JB, formatted_prev_date, trade_time_formatted, portfolio)
            isin_list = new_positions['InstrumentID'].to_list()
            for isin in isin_list:
                number_curr_day = fetch_transactions_current_day_Isin(conn_DB_JB, formatted_prev_date, trade_time_formatted, isin, portfolio)
                collateral_curr_day = fetch_collateral_current_day(conn_DB_JB, formatted_prev_date, trade_time_formatted, isin)
                currency = fetch_currency(conn_DB, isin)
                bpv = fetch_bpv(conn_DB, isin, trade_time_formatted)/100
                currency_multiplier = 7.45 if currency == 'EUR' else 1.0
                risk_inventory_isin = (number_curr_day+collateral_curr_day)*currency_multiplier
                inventory += risk_inventory_isin
                bpv_risk += risk_inventory_isin*bpv
                
            
            #Append values to original df
            df.at[index, 'inventoryRisk'] = (inventory)/1000
            df.at[index, 'bpv_risk'] = (bpv_risk)/1000

            #df_risk_inventory = pd.DataFrame(data) 

print(df.inventoryRisk)
print(df.bpv_risk)
#print(df_risk_inventory)

tradetime:  2024-10-03 23:00
prev date:  2024-10-02
0    1.120787e+07
Name: inventoryRisk, dtype: float64
0    67138.429927
Name: bpv_risk, dtype: float64


In [7]:
fb_27_06 = pd.read_csv('tmp2.tsv', sep='\t',header=None, decimal=',', thousands='.').iloc[:, [1, 3, 6]]
fb_27_06.to_csv('fb_27_06.csv')

In [8]:
#df_risk_inventory 

In [9]:
df_risk_inventory.to_csv('myrisk_27_06.csv')

In [10]:
df

,TradeKey,Result,OrderType,OrderStatus,Isin,Side,Price,Amount,TradeTime,DealersInCompetition,...,NumberTradesHour,NumberTradesDay,VolatilityWeek,VolatilityWeekNumber,VolatilityMonth,VolatilityMonthNumber,SentimentScore,inventory_Risk,bpv_risk,inventoryRisk
0,7249273161638215680,Lost,INQUIRY,TRADED_AWAY,DK0009531485,BUY,92.762,2000000.0,06/27/2023 10:02:46,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,34783.091893,8.475248e+06
